In [1]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/load_dicts.py

In [3]:
%run ../Python_files/util.py

In [4]:
# load logit_route_choice_probability_matrix
P = zload('../temp_files/logit_route_choice_probability_matrix.pkz')

In [5]:
import numpy as np
from numpy.linalg import inv

P = np.matrix(P)

In [6]:
np.size(P, 0), np.size(P, 1)

(56, 140)

In [7]:
import numpy as np
from numpy.linalg import inv

def samp_cov(x):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    K: number of samples
    S: sample covariance matrix
    ----------------
    return: inv(S)
    ----------------
    """
    x = np.matrix(x)
    K = np.size(x, 1)
    x_mean = sum(x[:,k] for k in range(K)) / K
    S = sum(np.dot(x[:,k] - x_mean, np.transpose(x[:,k] - x_mean)) for k in range(K)) / (K - 1)
    return S

In [8]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix.pkz')

In [9]:
# load link counts data

import json

with open('../temp_files/link_day_minute_Oct_dict_JSON.json', 'r') as json_file:
    link_day_minute_Oct_dict_JSON = json.load(json_file)

In [26]:
#weekend_Oct_list = [6, 7, 13, 14, 20, 21, 27, 28]
weekend_Oct_list = [7, 14, 21, 28]

In [27]:
link_day_minute_Oct_list = []
for link_idx in range(24):
    for day in weekend_Oct_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Oct_list.append(link_day_minute_Oct_dict_JSON[key] ['AM_flow_minute'][minute_idx])

In [28]:
len(link_day_minute_Oct_list)

11520

In [29]:
x = np.matrix(link_day_minute_Oct_list)
x = np.matrix.reshape(x, 24, 480)

x = np.nan_to_num(x)
y = np.array(np.transpose(x))
y = y[np.all(y != 0, axis=1)]
x = np.transpose(y)
x = np.matrix(x)

In [30]:
x[:,:2]

matrix([[ 1204.46020076,  1265.66641839],
        [ 1187.55975966,  1185.29839204],
        [ 1533.84659368,  1496.28684053],
        [ 1589.90512747,  1524.24404551],
        [ 1584.83465184,  1534.46713582],
        [ 1433.67503943,  1579.51219766],
        [ 1239.00540022,  1280.89491702],
        [ 1295.42342651,  1258.18843498],
        [ 1663.88318504,  1644.84677691],
        [ 1544.39468834,  1557.23349196],
        [ 1489.6766154 ,  1454.21826445],
        [ 1193.42136088,  1175.16455012],
        [ 1193.860984  ,  1187.56945656],
        [ 1240.18365593,  1269.89280973],
        [ 1170.27892676,  1216.94737784],
        [ 1101.89713326,   965.91134794],
        [ 1270.49441224,  1279.36847072],
        [ 1282.25644398,  1278.87414984],
        [ 1520.32015554,  1518.53668328],
        [ 1464.97917073,  1500.2901947 ],
        [ 1444.95856308,  1395.2222123 ],
        [ 1318.54752622,  1294.33427559],
        [ 1255.36117507,  1202.65278352],
        [ 1218.18133718,  1281.028

In [31]:
np.size(x, 0), np.size(x, 1)

(24, 480)

In [32]:
np.size(A,0), np.size(A,1)

(24, 140)

In [33]:
from gurobipy import *

L = 56  # dimension of lam

K = np.size(x, 1)
S = samp_cov(x)
inv_S = inv(S)

A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
# AP_t
AP_t = np.transpose(PA_t)

Q = np.dot(np.dot(PA_t, inv_S), AP_t)
b = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])


model = Model("OD_matrix_estimation")

lam = []
for l in range(L):
    lam.append(model.addVar(name='lam_' + str(l)))

model.update() 

# Set objective: (K/2) lam' * Q * lam - b * lam
obj = 0
for i in range(L):
    for j in range(L):
        obj += (1.0 /2) * K * lam[i] * Q[i, j] * lam[j]
for l in range(L):
    obj += - b[l] * lam[l]
model.setObjective(obj)

# Add constraint: lam >= 0
for l in range(L):
    model.addConstr(lam[l] >= 0)

model.update() 

model.optimize()

lam_list = []
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))
    lam_list.append(v.x)
print('Obj: %g' % obj.getValue())

Optimize a model with 56 rows, 56 columns and 56 nonzeros
Model has 1596 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [5e+00, 3e+03]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 56 rows and 0 columns
Presolve time: 0.00s
Presolved: 0 rows, 56 columns, 0 nonzeros
Presolved model has 1596 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 26
 AA' NZ     : 3.250e+02
 Factor NZ  : 3.510e+02
 Factor Ops : 6.201e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -2.77465584e+07  0.00000000e+00  7.30e+03 9.94e+02  1.00e+06     0s
   1  -5.60646623e+06 -8.68755271e+06  4.77e+02 6.49e+01  1.59e+05     0s
   2  -4.97355188e+06 -6.17679091e+06  4.77e-04 6.49e-05  2.15e+04     0s
   3  -5.18875554e+06 -5.29679432e+06  2.50e-06 3.40e-07  1.9

In [34]:
# write estimation result to file
n = 8  # number of nodes
with open('../temp_files/OD_demand_matrix_Oct_weekend.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1

In [35]:
lam_list

[3.11896123428376e-12,
 1116.3187749159435,
 3.453971146008826e-12,
 782.5489052207545,
 328.746636770248,
 3.4212069942166607,
 1.0423014652399637e-09,
 1.1510889550964305e-05,
 1.9801161182257477e-11,
 1179.3695831095083,
 1.9801161182900196e-11,
 1.9801161182513076e-11,
 1.980116118141127e-11,
 1.9914710094928362e-11,
 1833.9755431709434,
 189.51124653739294,
 0.008777520571285446,
 776.1357987153424,
 738.4705037215365,
 3.13528215038248,
 0.16892203179523335,
 582.9583292530399,
 695.0383473831,
 132.9270583954289,
 663.3782220024228,
 287.8794611057427,
 2.412882650185125,
 1219.9278795057112,
 0.011548227231361174,
 0.034064333527755064,
 0.01836545016032276,
 906.4105417777231,
 768.766630917698,
 904.4479864254079,
 1187.926392855803,
 8.871845608087814e-07,
 8.871935270670999e-07,
 8.871842018735691e-07,
 0.010562184241212691,
 732.1043944557383,
 930.0602247607085,
 0.07714412801662895,
 0.04503180112097316,
 454.4203835881315,
 0.098030649021986,
 0.008702824068021586,
 592